# **YouTube TF-IDF**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
# fasttext.util.download_model('ko', if_exists='ignore')   # FastText 모델 사용 시에만 필요
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

## **Read Data**

In [2]:
### YouTube Script
db = pymysql.connect(user='root',
                     passwd='0808',
                     host='127.0.0.1',
                     db='proj',
                     charset='utf8')

cursor = db.cursor(pymysql.cursors.DictCursor)

sql = "select * from youtube_suka_005930 where (length(date)=10)"
cursor.execute(sql)
result = cursor.fetchall()

# DataFrame으로 변경
suka = pd.DataFrame(result)

db.close()   # 메모리 절약

In [3]:
suka

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url
0,삼성전자,005930,슈카월드,2021081800,"개인투자자 수 약 570만 명, 국민주식 삼성전자의 하락","아[음악]wr우리 삼성전자가 능력 어빌 연속하락해 서희74,400원아유 분위가 3 ...",630644,1056,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=AoucusmMFx8
1,삼성전자,005930,슈카월드,2021050600,"박스권에 갇힌 삼성전자, 100조 투자와 어닝 서프라이즈 사이",으 out삼성전자가 1분기 실적 발표 했는데요 영업이 구조 삼척에 나왔습니다어닝 서...,403749,795,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=LbiU6yYWRCg
2,삼성전자,005930,슈카월드,2019110600,삼성전자 시스템 반도체에 133조원 투자! 무엇을 원하는가?,으 르3제 오늘의 메인 주제가 삼성 존재합니다 얘기를 왜 않은 양얼마전에 삼성전자 ...,786589,2239,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 슈카아재의\n경제/...",https://www.youtube.com/watch?v=s8iii0h5xoc
3,삼성전자,005930,슈카월드,2020111800,"삼성전자 역대 최고점 돌파, 가파른 원화강세와 몰리는 유동성",으 out화이자가 골은 아 107 되겠습니다 아 이거 다쳐 지난주에 세계를 강타한뉴...,297421,1581,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=aKv_BH5W5Yo
4,삼성전자,005930,슈카월드,2021020600,삼성전자의 역대급 배당과 애플의 진정한 경쟁자는 누구인가,으 out삼성전자가 배당을 발표했습니다 특별 배당 1932 원 예 총 13점을풀었는...,471414,1317,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=qVAZC5SNUSQ
...,...,...,...,...,...,...,...,...,...,...
443,삼성전자,005930,슈카월드,2019040300,현대차에 1조원 물리고 존버중인 엘리엇 근황,[음악]가장 유명한 얘기부터 드릴께요 이해를 좀 쉽게 해야 되니까 2003년에있었던...,663535,1624,엘리엇썰 1부 풀영상 바로가기↓\nhttps://www.youtube.com/wat...,https://www.youtube.com/watch?v=g8MJDRovq4g
444,삼성전자,005930,슈카월드,2019040600,"[1교시] 론스타, 외환은행으로 한국을 교육하다.",이 말은 수원을 짜서 돈을 쭉 걸어서 와 보니까 좀 멀고 먼 나라 꼴이아가 뭐한 되...,298268,1594,순이익이 '외환은행' \n우리나라 역사상 유례없는 참교육\n곧 2교시 올라갑니다. ...,https://www.youtube.com/watch?v=MV6ippM9HBw
445,삼성전자,005930,슈카월드,2019041800,복잡한 그리스 신화 쉽게 이해하기! - 기초편,[음악]그리스 신 합니다 그리스 나자의 줄이 뭐냐 넘을 수 없는 4차원의 벽 이미 ...,870596,3272,"2시간짜리 만담을 담백하게 55분으로 줄였습니다.\n\n어렵고 딱딱한 경제,시사,금...",https://www.youtube.com/watch?v=PENLh39FVw8
446,삼성전자,005930,슈카월드,2019022300,[꿀잼 세계사] 멕시코 비극의 원인. 마약왕과 카르텔들의 역사 - 슈카월드 아재토...,지금 이제 이걸 않았죠 마약류 에정의를 대충 않았습니다 자정 1 않았어야 안개비가 ...,1650670,4953,"1부 : 마약의 종류와 기분 변화(효과) \n2부 : 잔인한 마약 유통역사와 ""진짜...",https://www.youtube.com/watch?v=suF_Wh-kjbc


In [5]:
tfidf_vect = TfidfVectorizer(max_features=100)
tdm = tfidf_vect.fit_transform(suka['text'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=500, stop_words='english')
tdm = tfidf.fit_transform(df[0])

In [ ]:
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})